<a href="https://colab.research.google.com/github/IsmaelArista/Global-Hitss/blob/main/Examen_CursoIA_AristaRomeroJuanIsmael.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_classification

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.metrics import classification_report, make_scorer
from imblearn.metrics import geometric_mean_score, classification_report_imbalanced
from sklearn.model_selection import  cross_validate,  RepeatedStratifiedKFold

In [27]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"

# Leer el dataset
data = pd.read_csv(url, delim_whitespace=True, header=None)
data.columns = [
    "status", "duration", "credit_history", "purpose", "amount",
    "savings", "employment_duration", "installment_rate", "personal_status_sex",
    "other_debtors", "present_residence", "property", "age",
    "other_installment_plans", "housing", "number_credits", "job",
    "people_liable", "telephone", "foreign_worker", "credit_risk"
]

print("Dimensiones del dataset:", data.shape)
print("\nPrimeras 5 filas:")
print(data.head())

print("\nInformación general del dataset:")
print(data.info())

print("\nConteo de valores nulos por columna:")
print(data.isnull().sum())

print("\nDistribución de la variable objetivo credit_risk:")
print(data['credit_risk'].value_counts())

print("\nProporciones de la variable objetivo credit_risk:")
print(data['credit_risk'].value_counts(normalize=True))


/tmp/ipython-input-351687017.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(url, delim_whitespace=True, header=None)


Dimensiones del dataset: (1000, 21)

Primeras 5 filas:
  status  duration credit_history purpose  amount savings employment_duration  \
0    A11         6            A34     A43    1169     A65                 A75   
1    A12        48            A32     A43    5951     A61                 A73   
2    A14        12            A34     A46    2096     A61                 A74   
3    A11        42            A32     A42    7882     A61                 A74   
4    A11        24            A33     A40    4870     A61                 A73   

   installment_rate personal_status_sex other_debtors  ...  property age  \
0                 4                 A93          A101  ...      A121  67   
1                 2                 A92          A101  ...      A121  22   
2                 2                 A93          A101  ...      A121  49   
3                 2                 A93          A103  ...      A122  45   
4                 3                 A93          A101  ...      A124  53   

 

In [28]:
# Separar fX y Y
X = data.drop(columns=['credit_risk'])
y = data['credit_risk']

# Partición 85%/15%
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    stratify=y,
    random_state=42
)

X_train = X_train.reset_index(drop=True)
X_test  = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test  = y_test.reset_index(drop=True)

print("Tamaños:")
print("  X_train:", X_train.shape, "  y_train:", y_train.shape)
print("  X_test: ", X_test.shape,  "  y_test: ", y_test.shape)

print("\nProporción de clases en todo el dataset:")
print(y.value_counts(normalize=True))


Tamaños:
  X_train: (850, 20)   y_train: (850,)
  X_test:  (150, 20)   y_test:  (150,)

Proporción de clases en todo el dataset:
credit_risk
1    0.7
2    0.3
Name: proportion, dtype: float64


# Pipeline

In [30]:
# Cuantitativas
quantitative = ['duration', 'amount', 'age']

# Ordinales
ordinal = ['installment_rate', 'present_residence', 'number_credits', 'people_liable']

# Categóricas
categorical = [
    'status', 'credit_history', 'purpose', 'savings',
    'employment_duration', 'personal_status_sex', 'other_debtors',
    'property', 'other_installment_plans', 'housing', 'job',
    'telephone', 'foreign_worker'
]



# Cuantitativas
quantitative_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Ordinales:
ordinal_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Categóricas con binarias
categorical_pipe = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Se une todo
preprocess = ColumnTransformer(
    transformers=[
        ('num', quantitative_pipe, quantitative),
        ('ord', ordinal_pipe, ordinal),
        ('cat', categorical_pipe, categorical)
    ],
)

print("Cuantitativas escaladas:", quantitative)
print("Ordinales (sin escalar):", ordinal)
print("Categóricas (one-hot):", categorical)


Cuantitativas escaladas: ['duration', 'amount', 'age']
Ordinales (sin escalar): ['installment_rate', 'present_residence', 'number_credits', 'people_liable']
Categóricas (one-hot): ['status', 'credit_history', 'purpose', 'savings', 'employment_duration', 'personal_status_sex', 'other_debtors', 'property', 'other_installment_plans', 'housing', 'job', 'telephone', 'foreign_worker']


Justificación:
Las variables numéricas usan la mediana porque es robusta a valores extremos.

Las variables categóricas y binarias usan el valor más frecuente porque no tiene sentido calcular medias en valores no numéricos.

Las ordinales también usan mediana para mantener el orden pero sin escalar aún.

In [31]:
def confusion_counts(y_true, y_pred, pos_label=1):

    # Aseguramos el orden [pos, neg] para extraer bien TP/TN/FP/FN
    labels = [pos_label, 2 if pos_label == 1 else 1]
    cm = confusion_matrix(y_true, y_pred, labels=labels)

    TP = cm[0, 0]
    FN = cm[0, 1]
    FP = cm[1, 0]
    TN = cm[1, 1]
    return TP, TN, FP, FN

def accuracy_score_manual(y_true, y_pred, pos_label=1):
    TP, TN, FP, FN = confusion_counts(y_true, y_pred, pos_label)
    total = TP + TN + FP + FN
    return (TP + TN) / total if total > 0 else 0.0

def precision_score_manual(y_true, y_pred, pos_label=1):
    TP, TN, FP, FN = confusion_counts(y_true, y_pred, pos_label)
    denom = TP + FP
    return TP / denom if denom > 0 else 0.0

def recall_score_manual(y_true, y_pred, pos_label=1):
    TP, TN, FP, FN = confusion_counts(y_true, y_pred, pos_label)
    denom = TP + FN
    return TP / denom if denom > 0 else 0.0

def f1_score_manual(y_true, y_pred, pos_label=1):
    prec = precision_score_manual(y_true, y_pred, pos_label)
    rec  = recall_score_manual(y_true, y_pred, pos_label)
    denom = prec + rec
    return 2 * prec * rec / denom if denom > 0 else 0.0

def gmean_score_manual(y_true, y_pred, pos_label=1):
    """
    G-Mean = sqrt(Sensibilidad * Especificidad)
    Sensibilidad = Recall positivo
    Especificidad = TN / (TN + FP)
    """
    TP, TN, FP, FN = confusion_counts(y_true, y_pred, pos_label)
    sens = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    espec = TN / (TN + FP) if (TN + FP) > 0 else 0.0
    return np.sqrt(sens * espec)

def compute_all_metrics(y_true, y_pred, pos_label=1):
    """Devuelve un dict con todas las métricas."""
    return {
        "accuracy": accuracy_score_manual(y_true, y_pred, pos_label),
        "precision": precision_score_manual(y_true, y_pred, pos_label),
        "recall": recall_score_manual(y_true, y_pred, pos_label),
        "f1": f1_score_manual(y_true, y_pred, pos_label),
        "gmean": gmean_score_manual(y_true, y_pred, pos_label),
    }

print("Funciones de métricas listas (accuracy, precision, recall, f1, gmean).")


Funciones de métricas listas (accuracy, precision, recall, f1, gmean).


In [33]:
log_reg = LogisticRegression()

# Pipeline con modelo
pipe = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", log_reg)
])

# Val. cruzada estratificada: 5 particiones, 3 repeticiones
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)

resultados = []          # guardamos métricas de cada fold
recalls_train = []       # boxplot train
recalls_val = []         # boxplot validación

fold_id = 1
for train_idx, val_idx in cv.split(X_train, y_train):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

    # Entrenar en el fold
    pipe.fit(X_tr, y_tr)

    # Predicciones
    y_tr_pred = pipe.predict(X_tr)
    y_val_pred = pipe.predict(X_val)

    # Métricas (clase positiva = 1)
    m_tr  = compute_all_metrics(y_tr,  y_tr_pred,  pos_label=1)
    m_val = compute_all_metrics(y_val, y_val_pred, pos_label=1)

    # Guardar para resumen
    resultados.append({
        "fold": fold_id,
        "acc_train": m_tr["accuracy"], "prec_train": m_tr["precision"],
        "rec_train": m_tr["recall"],   "f1_train":   m_tr["f1"], "gmean_train": m_tr["gmean"],
        "acc_val":  m_val["accuracy"], "prec_val":  m_val["precision"],
        "rec_val":  m_val["recall"],   "f1_val":    m_val["f1"], "gmean_val":  m_val["gmean"],
    })
    recalls_train.append(m_tr["recall"])
    recalls_val.append(m_val["recall"])

    fold_id += 1

# DataFrame con todos los folds
df_res = pd.DataFrame(resultados)

print("\nResumen por fold (primeras filas):")
print(df_res.head())

# Promedios y desviaciones
promedios = df_res.mean(numeric_only=True)
desvios   = df_res.std(numeric_only=True)

print("\nPromedios (validación):")
print(promedios[["acc_val","prec_val","rec_val","f1_val","gmean_val"]])

print("\nDesviaciones estándar (validación):")
print(desvios[["acc_val","prec_val","rec_val","f1_val","gmean_val"]])

# Guardamos los recalls
recall_train_folds = np.array(recalls_train)
recall_val_folds   = np.array(recalls_val)

print("\nListos los vectores para boxplot de Recall (train y validación).")
print("Tamaño recall_train_folds:", recall_train_folds.shape)
print("Tamaño recall_val_folds:  ", recall_val_folds.shape)



Resumen por fold (primeras filas):
   fold  acc_train  prec_train  rec_train  f1_train  gmean_train   acc_val  \
0     1   0.783824    0.812144   0.899160  0.853440     0.680296  0.776471   
1     2   0.792647    0.815443   0.909664  0.859980     0.687509  0.752941   
2     3   0.794118    0.819392   0.905462  0.860279     0.695558  0.717647   
3     4   0.801471    0.829787   0.901261  0.864048     0.715878  0.717647   
4     5   0.780882    0.809074   0.899160  0.851741     0.673786  0.752941   

   prec_val   rec_val    f1_val  gmean_val  
0  0.779310  0.949580  0.856061   0.594782  
1  0.793893  0.873950  0.832000   0.641304  
2  0.808696  0.781513  0.794872   0.666625  
3  0.770992  0.848739  0.808000   0.591169  
4  0.808000  0.848739  0.827869   0.670323  

Promedios (validación):
acc_val      0.745490
prec_val     0.791669
rec_val      0.864986
f1_val       0.826016
gmean_val    0.632616
dtype: float64

Desviaciones estándar (validación):
acc_val      0.023368
prec_val     0.0

Perdón, entre a la clase 11:35 :(